In [1]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from utils import load_model_and_files, get_model_details, reconstruct_word, merge_results, preprocess_text, preprocess_stopwords, preprocess_lemmatization, traduci_output, process_text_with_models, process_emotions_and_translate

In [2]:
# 1. Recupera i dettagli dei modelli
model_details = get_model_details()

# 2. Assegna manualmente le tuple a  separate
(model_1_id, model_1_type, model_1_files), (model_2_id, model_2_type, model_2_files), (model_3_id, model_3_type, model_3_files), (model_4_id, model_4_type, model_4_files) = model_details

# 3. Usa i dettagli per caricare i modelli
print(f"Caricamento modello 1: {model_1_id}")
model_1_pipeline = load_model_and_files(model_1_id, model_type=model_1_type)

print(f"Caricamento modello 2: {model_2_id}")
model_2_pipeline = load_model_and_files(model_2_id, model_type=model_2_type)

print(f"Caricamento modello 3: {model_3_id}")
model_3_pipeline = load_model_and_files(model_3_id, model_type=model_3_type)

print(f"Caricamento modello 4: {model_4_id}")
model_4_pipeline = load_model_and_files(model_4_id, model_type=model_4_type)


Caricamento modello 1: osiria/bert-italian-uncased-ner


Device set to use cpu


Caricamento modello 2: IVN-RIN/MedPsyNIT


Device set to use cpu


Caricamento modello 3: SamLowe/roberta-base-go_emotions


Device set to use cpu


Caricamento modello 4: Helsinki-NLP/opus-mt-it-en


c:\Users\lore9\Documents\GitHub\data_semantics\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
from pymongo import MongoClient

# Connetti al database MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["data_semantics_prova_6"]
collection = db["post"]
autori_collection = db["autori"]  # Collezione che contiene gli autori

DIMENSIONE_BLOCCO = 50

def recupera_autori():
    # Recupera la lista di autori dal database
    autori_cursor = autori_collection.find()
    autori = [autore["author"] for autore in autori_cursor]  # Supponendo che il campo del nome dell'autore sia "nome"
    return autori

def analisi_semantica(text, autori):
    # Supponendo che i risultati siano già ottenuti
    risultato_ner = process_text_with_models(text, model_1_pipeline, model_2_pipeline, model_1_id, model_2_id)
    risultato_sentiment = process_emotions_and_translate(text, model_4_pipeline, model_3_pipeline)[0]

    # Filtrare i sentimenti per raggiungere uno score cumulativo di almeno 0.95
    selected_sentiments = []
    cumulative_score = 0

    for sentiment in risultato_sentiment:
        if cumulative_score >= 0.95:
            break
        selected_sentiments.append(sentiment)
        cumulative_score += sentiment['score']

    # Controlla se la lista non è vuota
    if risultato_ner:
        # Itera sulla lista e converte i punteggi 'score' in float64
        for item in risultato_ner:
            item["score"] = float(item["score"])  # Converte np.float32 in float64
            # Se l'entità è un autore, etichettala come "PER"
            if item["word"] in autori:
                item["label"] = "PER"
    else:
        print("La lista 'ner' è vuota!")

    risultato_sentiment = {item['label']: item['score'] for item in risultato_sentiment}

    selected_sentiments = {item['label']: item['score'] for item in selected_sentiments}

    # Creazione del dizionario
    data_semantic_analysis = {
        "ner": risultato_ner,
        "sentiment_analysis_relevant": selected_sentiments,
        "sentiment_analysis_full": risultato_sentiment
    }
    return data_semantic_analysis

# Recupera la lista di autori una volta all'inizio
autori = recupera_autori()

In [14]:
# Collega MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["data_semantics_prova_6"]
collection = db["post"]

DIMENSIONE_BLOCCO = 50


cursor = collection.find().batch_size(DIMENSIONE_BLOCCO)
for documento in cursor:
    try:
        messaggio = documento.get("message")
        if not messaggio:
            continue  # Salta documenti senza messaggio
        print(messaggio)
        
        # Esegui la pipeline
        risultato = analisi_semantica(messaggio,autori)
        
        # Aggiorna il documento con i risultati
        collection.update_one(
            {"_id": documento["_id"]},  # Filtro per identificare il documento
            {"$set": risultato}         # Aggiungi i risultati come nuove chiavi
        )

    except Exception as e:
        print(f"Errore nel processare il documento {documento['_id']}: {e}")

print("Elaborazione completata!")

Ciao ragazze! Sono nuova qui e da pochissimo ho iniziato un percorso di PMA eterologa dopo PRP ovarico utile al ringiovanimento ovarico. Qualcun'altra coraggiosa "vecchina" come me? :grinning_face:
La lista 'ner' è vuota!
QUOTE (Sarengi78 @ 27/1/2024, 18:00)  Ciao e benvenuta! Dice sei seguita e se posso dove hai fatto il PRP? È un tema di cui abbiamo parlato raramente qui è il tuo contributo sarebbe una grande informazione
Ciao! Innanzitutto ho fatto un mezzo casino...sto facendo omologa non eterologa come ho scritto nel titolo...manmaggia l'ansia che ho in qst gg! Proprio vista la mia età e la mia assoluta voglia di non andare in Ovodonazione il dottore di Atene che mi segue mi ha proposto prp. Dopo un mese dal PRP i miei valori sono migliorati di circa un 30% e quindi siamo subito partiti con la stimolazione e picking. Ora sono in attesa di vedere se gli embrioni arrivano a blastocisti, oggi è il 5o e nom ci siamo ancora. Mi dicono di attendere domani... ho un po di seria ansia
In I